#Lectura del dataset

https://spacy.io/models/ru

In [1]:
#%%capture
import pandas as pd
import nltk
!pip install -U spacy
!pip install --upgrade spacy
import spacy
import numpy as np
import scipy as sp
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize

nltk.download('stopwords')
stop_words = stopwords.words('russian')
nltk.download('punkt')
!python -m spacy download ru_core_news_md
nlp = spacy.load('ru_core_news_md')
import string
puntuacion = list(string.punctuation)
puntuacion.append(" ")

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

!pip install emoji_extractor
from emoji_extractor.extract import Extractor
import re

# Download emoji sentiment
!wget https://www.clarin.si/repository/xmlui/handle/11356/1048/allzip
!unzip allzip
emoji_sent_dict = pd.read_csv("Emoji_Sentiment_Data_v1.0.csv", sep = ",")

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix
from sklearn.metrics import classification_report
import scipy as sp
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import chi2
from sklearn.model_selection import StratifiedKFold

     |████████████████████████████████| 6.0 MB 4.4 MB/s 
     |████████████████████████████████| 628 kB 50.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 181 kB 44.9 MB/s 
     |████████████████████████████████| 451 kB 49.4 MB/s 
     |████████████████████████████████| 10.1 MB 16.5 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4
[nltk_data] Downloading package stopwords to /

In [2]:
from google.colab import drive
drive.mount('/content/drive')

directorio = "/content/drive/MyDrive/TFM Big Data & BA/Propuestas/Proyecto Final/0 - Dataset/train_data.csv"
df = pd.read_csv(directorio)

Mounted at /content/drive


,Unnamed: 0,de,de_processed,normaliza
0,0,Продам СРОЧНО/\nГарнитура - Динамики для телеф...,Продам СРОЧНО/\nГарнитура - Динамики для телеф...,продать гарнитура динамика китай оригинал подк...
1,1,Зарядное устройство подойдет для любого соврем...,Зарядное устройство подойдет для любого соврем...,зарядный устройство подойти любой современный ...
2,2,"Отличный крепкий чехол для защиты от ударов, о...","Отличный крепкий чехол для защиты от ударов, о...",отличный крепкий чехол защита удар отлично под...
3,3,Продам дисплей на смартфон IPhone 5/\nТорг/\nП...,Продам дисплей на смартфон IPhone 5/\nТорг/\nП...,продать дисплей смартфон iphone покупка штука ...
4,4,Аккумулятор для Samsung Galaxy Star Plus Duos ...,Аккумулятор для Samsung Galaxy Star Plus Duos ...,аккумулятор samsung galaxy star plus duos ориг...
...,...,...,...,...
208001,622753,"Состояние новой ,в юбилейном стоит 750р,продаю...","Состояние новой ,в юбилейном стоит 750р,продаю...",состояние новый юбилейном стоить 750р продават...
208002,622754,Нет одной струны,Нет одной струны,одной струна
208003,622756,Продам педаль эффекта Distortion. Полное назва...,Продам педаль эффекта Distortion. Полное назва...,продать педаль эффект distortion полный назван...
208004,622757,Использовался минимально и только в домашних у...,Использовался минимально и только в домашних у...,использоваться минимально домашний условие сос...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df=df[df["sold"]==1]

In [3]:
descr = df[['description']]
descr.columns = ['de']
title = df[['title']]
title.columns = ['ti']

descr = descr[descr.de.notnull()]
title = title[title.ti.notnull()]

print("We've divided the dataset in 2:")
print(" - descr: Description's data. It has", len(descr), "values.")
print(" - title: Title's data. It has", len(title), "values.")

We've divided the dataset in 2:
 - descr: Description's data. It has 622762 values.
 - title: Title's data. It has 622762 values.


In [5]:
title

,ti
0,Nokia MD - 8 (Original) Black Edition
1,Автомобильное зарядное устройство USB 1А 2А
2,Чехол для meizu m5 note
3,Дисплей на смартфон iPhone 5
4,Аккумулятор Samsung Galaxy Star Plus Duos GT-S...
...,...
622757,Комбоусилитель Roland Cube 20X
622758,Чехол на гитару
622759,Ремень для гитары
622760,Педали эффектов


#`description`

In [6]:
%%capture
# En primer lugar limpiamos los emojis de las descripciones
def limpia_emoji(text):
    emoji_pattern = re.compile('['
        u'\U0001F300-\U0001F6FF' u'\U0001F1E0-\U0001F1FF' u'\u2600-\u26FF\u2700-\u27BF'
        u'\u2934' u'\u2935' u'\u2B05' u'\u2B06' u'\u2B07' u'\u2B1C' u'\u2B1B'
        u'\u2B50' u'\u2B55' u'\u3030' u'\u303D' u'\u3297' u'\u3299' u'\u00A9'
        u'\u00AE' u'\u203C' u'\u2049' u'\u2122' u'\u2139' u'\u2194-\u2199' 
        u'\u21A9' u'\u21AA' u'\u231A' u'\u231B' u'\u2328' u'\u23CF'
        u'\u23E9-\u23F3' u'\u23F8' u'\u23F9' u'\u23FA' u'\u24C2' u'\u25AA'
        u'\u25AB' u'\u25B6' u'\u25C0' u'\u25FB' u'\u25FD' u'\u25FC' u'\u25FE'
        ']+', flags = re.UNICODE)
    str2 = re.sub(emoji_pattern, r' ', text)
    return str2

descr['de_processed'] = descr['de'].apply(lambda x: limpia_emoji(x))

In [7]:
# Eliminar espacios
def eliminar_espacios(text): 
    return  " ".join(text.split()) 

# To lower
def texto_to_lower(text):
  return text.lower()

# Tokenizador
def tokenization(text):
  tokens = word_tokenize(text)
  return tokens

# Quitar stop words
def quitar_stopwords(tokens):
    filtered_sentence = [w for w in tokens if not w in stop_words]
    return filtered_sentence

# Eliminar signos de puntuación
def quitar_puntuacion(tokens):
    words=[word for word in tokens if word.isalnum()]
    return words

# Lemmatization
def lematizar(tokens):
    sentence = " ".join(tokens)
    mytokens = nlp(sentence)
    # Lematizamos los tokens y los convertimos  a minusculas
    mytokens = [ word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Extraemos el text en una string
    return " ".join(mytokens)

def quitar_numeros(tokens):
  sin_numeros=[w for w in tokens if w.isnumeric()==False]
  return sin_numeros

In [ ]:
# Eliminamos espacios
descr["normaliza"] = descr["de_processed"].apply(lambda x: eliminar_espacios(x))
# Pasamos a minúscula
descr["normaliza"] = descr["normaliza"].apply(lambda x: texto_to_lower(x))
# Tokenizar
descr["normaliza"] = descr["normaliza"].apply(lambda x: tokenization(x))
# Quitar Stopwords
descr["normaliza"] = descr["normaliza"].apply(lambda x: quitar_stopwords(x))
# Quitar puntuación
descr["normaliza"] = descr["normaliza"].apply(lambda x: quitar_puntuacion(x))
# Quitar números
descr["normaliza"] = descr["normaliza"].apply(lambda x: quitar_numeros(x))
# Lematizamos
descr["normaliza"] = descr["normaliza"].apply(lambda x: lematizar(x))

In [ ]:
descr

,de,de_processed,normaliza
0,Продам СРОЧНО/\nГарнитура - Динамики для телеф...,Продам СРОЧНО/\nГарнитура - Динамики для телеф...,продать гарнитура динамика китай оригинал подк...
1,Зарядное устройство подойдет для любого соврем...,Зарядное устройство подойдет для любого соврем...,зарядный устройство подойти любой современный ...
2,"Отличный крепкий чехол для защиты от ударов, о...","Отличный крепкий чехол для защиты от ударов, о...",отличный крепкий чехол защита удар отлично под...
3,Продам дисплей на смартфон IPhone 5/\nТорг/\nП...,Продам дисплей на смартфон IPhone 5/\nТорг/\nП...,продать дисплей смартфон iphone покупка штука ...
4,Аккумулятор для Samsung Galaxy Star Plus Duos ...,Аккумулятор для Samsung Galaxy Star Plus Duos ...,аккумулятор samsung galaxy star plus duos ориг...
...,...,...,...
622753,"Состояние новой ,в юбилейном стоит 750р,продаю...","Состояние новой ,в юбилейном стоит 750р,продаю...",состояние новый юбилейном стоить 750р продават...
622754,Нет одной струны,Нет одной струны,одной струна
622756,Продам педаль эффекта Distortion. Полное назва...,Продам педаль эффекта Distortion. Полное назва...,продать педаль эффект distortion полный назван...
622757,Использовался минимально и только в домашних у...,Использовался минимально и только в домашних у...,использоваться минимально домашний условие сос...


In [ ]:
from google.colab import files
descr.to_csv('descr.csv')
files.download('descr.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Hallamos la longitud de caracteres y de tokens por decripción
lc = list()
lto = list()

for lp in depurado:
  # Longitud de caracteres:
  lc.append(sum(list(map(lambda x: len(x), lp))))
  # Longitud de tokens:
  lto.append(len(lp))

descr['long_caracteres'] = lc
descr['long_tokens'] = lto

#`title`

In [8]:
%%capture
# En primer lugar limpiamos los emojis de las descripciones
def limpia_emoji(text):
    emoji_pattern = re.compile('['
        u'\U0001F300-\U0001F6FF' u'\U0001F1E0-\U0001F1FF' u'\u2600-\u26FF\u2700-\u27BF'
        u'\u2934' u'\u2935' u'\u2B05' u'\u2B06' u'\u2B07' u'\u2B1C' u'\u2B1B'
        u'\u2B50' u'\u2B55' u'\u3030' u'\u303D' u'\u3297' u'\u3299' u'\u00A9'
        u'\u00AE' u'\u203C' u'\u2049' u'\u2122' u'\u2139' u'\u2194-\u2199' 
        u'\u21A9' u'\u21AA' u'\u231A' u'\u231B' u'\u2328' u'\u23CF'
        u'\u23E9-\u23F3' u'\u23F8' u'\u23F9' u'\u23FA' u'\u24C2' u'\u25AA'
        u'\u25AB' u'\u25B6' u'\u25C0' u'\u25FB' u'\u25FD' u'\u25FC' u'\u25FE'
        ']+', flags = re.UNICODE)
    str2 = re.sub(emoji_pattern, r' ', text)
    return str2

title['ti_processed'] = title['ti'].apply(lambda x: limpia_emoji(x))

In [9]:
# Eliminar espacios
def eliminar_espacios(text): 
    return  " ".join(text.split()) 

# To lower
def texto_to_lower(text):
  return text.lower()

# Tokenizador
def tokenization(text):
  tokens = word_tokenize(text)
  return tokens

# Quitar stop words
def quitar_stopwords(tokens):
    filtered_sentence = [w for w in tokens if not w in stop_words]
    return filtered_sentence

# Eliminar signos de puntuación
def quitar_puntuacion(tokens):
    words=[word for word in tokens if word.isalnum()]
    return words

# Lemmatization
def lematizar(tokens):
    sentence = " ".join(tokens)
    mytokens = nlp(sentence)
    # Lematizamos los tokens y los convertimos  a minusculas
    mytokens = [ word.lemma_ if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    # Extraemos el text en una string
    return " ".join(mytokens)

def quitar_numeros(tokens):
  sin_numeros=[w for w in tokens if w.isnumeric()==False]
  return sin_numeros

In [10]:
# Eliminamos espacios
title["normaliza"] = title["ti_processed"].apply(lambda x: eliminar_espacios(x))
# Pasamos a minúscula
title["normaliza"] = title["normaliza"].apply(lambda x: texto_to_lower(x))
# Tokenizar
title["normaliza"] = title["normaliza"].apply(lambda x: tokenization(x))
# Quitar Stopwords
title["normaliza"] = title["normaliza"].apply(lambda x: quitar_stopwords(x))
# Quitar puntuación
title["normaliza"] = title["normaliza"].apply(lambda x: quitar_puntuacion(x))
# Quitar números
title["normaliza"] = title["normaliza"].apply(lambda x: quitar_numeros(x))
# Lematizamos
title["normaliza"] = title["normaliza"].apply(lambda x: lematizar(x))

In [13]:
title

,ti,ti_processed,normaliza
0,Nokia MD - 8 (Original) Black Edition,Nokia MD - 8 (Original) Black Edition,nokia md original black edition
1,Автомобильное зарядное устройство USB 1А 2А,Автомобильное зарядное устройство USB 1А 2А,автомобильный зарядный устройство usb 1а 2а
2,Чехол для meizu m5 note,Чехол для meizu m5 note,чехол meizu m5 note
3,Дисплей на смартфон iPhone 5,Дисплей на смартфон iPhone 5,дисплей смартфон iphone
4,Аккумулятор Samsung Galaxy Star Plus Duos GT-S...,Аккумулятор Samsung Galaxy Star Plus Duos GT-S...,аккумулятор samsung galaxy star plus duos
...,...,...,...
622757,Комбоусилитель Roland Cube 20X,Комбоусилитель Roland Cube 20X,комбоусилитель roland cube 20x
622758,Чехол на гитару,Чехол на гитару,чехол гитара
622759,Ремень для гитары,Ремень для гитары,ремень гитара
622760,Педали эффектов,Педали эффектов,педаль эффект


In [12]:
#Descargamos el dataset train nuevo
from google.colab import files
title.to_csv('titulos.csv')
files.download('titulos.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Hallamos la longitud de caracteres y de tokens por decripción
lc = list()
lto = list()

for lp in depurado:
  # Longitud de caracteres:
  lc.append(sum(list(map(lambda x: len(x), lp))))
  # Longitud de tokens:
  lto.append(len(lp))

title['long_caracteres'] = lc
title['long_tokens'] = lto